In [22]:
import scipy.io.idl as idl
import numpy as np
from scipy.interpolate import griddata

In [48]:
from pyvisfile.vtk import write_structured_grid

import numpy as np

angle_mesh = np.mgrid[1:2:10j, 0:2*np.pi:20j, 0:np.pi:30j]

r = angle_mesh[0, np.newaxis]
phi = angle_mesh[1, np.newaxis]
theta = angle_mesh[2, np.newaxis]
mesh = np.vstack((
    r*np.sin(theta)*np.cos(phi),
    r*np.sin(theta)*np.sin(phi),
    r*np.cos(theta),
    ))

from pytools.obj_array import make_obj_array
vec = make_obj_array([
    np.sin(theta)*np.cos(phi),
    np.sin(theta)*np.sin(phi),
    np.cos(theta),
    ])

print 'mesh', mesh.shape
print 'data', phi.shape
print 'r', r.shape

write_structured_grid("yo1.vts", mesh,
        point_data=[("phi", phi), ("vec", vec)])

mesh (3, 10, 20, 30)
data (1, 10, 20, 30)
r (1, 10, 20, 30)


RuntimeError: output file 'yo1.vts' already exists

# Try to store a single time from Jason's IDL scripts
Store $\vec{B}$, $T$, and $n$.

In [12]:
b_x_measurement_points_timestamp = '115110949362'
b_y_measurement_points_timestamp = '115110949522'
b_z_measurement_points_timestamp = '115110938869'
triple_measurement_points_timestamp = '115112535917'
b_x_measurements_timestamp = '115112539367'
b_y_measurements_timestamp = '115112539384'
b_z_measurements_timestamp = '115112539331'
n_measurements_timestamp = '115112539443'
t_measurements_timestamp = '115112539427'

In [13]:
data_path = '../../cataloging_physics_quantities/output/intermediate/'
data_ending = '.sav'
b_x_measurement_points_file = data_path + b_x_measurement_points_timestamp + data_ending
b_y_measurement_points_file = data_path + b_y_measurement_points_timestamp + data_ending
b_z_measurement_points_file = data_path + b_z_measurement_points_timestamp + data_ending
triple_measurement_points_file = data_path + triple_measurement_points_timestamp + data_ending 
b_x_measurements_file = data_path + b_x_measurements_timestamp + data_ending
b_y_measurements_file = data_path + b_y_measurements_timestamp + data_ending
b_z_measurements_file = data_path + b_z_measurements_timestamp + data_ending
n_measurements_file = data_path + n_measurements_timestamp + data_ending
t_measurements_file = data_path + t_measurements_timestamp + data_ending

In [14]:
b_x_measurement_points = idl.readsav(b_x_measurement_points_file)
b_y_measurement_points = idl.readsav(b_y_measurement_points_file)
b_z_measurement_points = idl.readsav(b_z_measurement_points_file)
triple_measurement_points =  idl.readsav(triple_measurement_points_file)
b_x_measurements = idl.readsav(b_x_measurements_file)
b_y_measurements = idl.readsav(b_y_measurements_file)
b_z_measurements = idl.readsav(b_z_measurements_file)
n_measurements = idl.readsav(n_measurements_file)
t_measurements = idl.readsav(t_measurements_file)

In [20]:
def read_data(file_name):
    r"""
    Read IDL sav file.
    """
    data_dict = idl.readsav(file_name)    
    return data_dict

In [132]:
def resample_on_structutred_grid(data_dict, 
                                 time_point,
                                 x_min, x_max,
                                 y_min, y_max,
                                 x_points=50, 
                                 y_points=50, 
                                 to_clip=[0, -1, 0, -1]):
    r"""
    """
    x_points = np.linspace(x_min, x_max, 100)
    y_points = np.linspace(y_min, y_max, 100)
    x_grid, y_grid = np.meshgrid(x_points, y_points)
    quantity_interpolated = griddata(np.dstack((data_dict['x_out'], 
                                                data_dict['y_out']))[0],
                                                data_dict['a_out'][time_point],
                                                (x_grid, y_grid))
    x_slice = slice(to_clip[0], to_clip[1])
    y_slice = slice(to_clip[2], to_clip[3])
    quantity_interpolated = quantity_interpolated[x_slice, y_slice]
    x_grid = x_grid[x_slice, y_slice]
    y_grid = y_grid[x_slice, y_slice]
    return quantity_interpolated, x_grid, y_grid

In [ ]:
def determine_scalar_sample_bounds(data_dicts):
    x_mins = np.asarray([np.nanmin(data_dict['x_out']) for data_dict in data_dicts])
    x_maxs = np.asarray([np.nanmax(data_dict['x_out']) for data_dict in data_dicts])  
    y_mins = np.asarray([np.nanmin(data_dict['y_out']) for data_dict in data_dicts])
    y_maxs = np.asarray([np.nanmax(data_dict['y_out']) for data_dict in data_dicts])
    x_min = x_mins.max()
    x_max = x_maxs.min()
    y_min = y_mins.max()
    y_max = y_maxs.min()
    return x_min, x_max, y_min, y_max    

In [ ]:
def resample_on_structutred_grid(data_dict, 
                                 time_point, 
                                 x_points=50, 
                                 y_points=50, 
                                 to_clip=[0, -1, 0, -1]):
    r"""
    """
    quantity_max = data_dict['a_out'].max()
    quantity_min = data_dict['a_out'].min()
    x_min = np.nanmin(data_dict['x_out'])
    x_max = np.nanmax(data_dict['x_out'])    
    y_min = np.nanmin(data_dict['y_out'])
    y_max = np.nanmax(data_dict['y_out'])
    x_points = np.linspace(x_min, x_max, 100)
    y_points = np.linspace(y_min, y_max, 100)
    x_grid, y_grid = np.meshgrid(x_points, y_points)
    quantity_interpolated = griddata(np.dstack((data_dict['x_out'], 
                                                data_dict['y_out']))[0],
                                                data_dict['a_out'][time_point],
                                                (x_grid, y_grid))
    x_slice = slice(to_clip[0], to_clip[1])
    y_slice = slice(to_clip[2], to_clip[3])
    quantity_interpolated = quantity_interpolated[x_slice, y_slice]
    x_grid = x_grid[x_slice, y_slice]
    y_grid = y_grid[x_slice, y_slice]
    return quantity_interpolated, x_grid, y_grid

In [57]:
def prepare_mesh(x_grid, y_grid, z_position):
    r"""
    """
    shape = (x_grid.shape[0], x_grid.shape[1])
    z_grid = np.ones(shape)*z_position
    x_grid = np.expand_dims(np.expand_dims(x_grid, 2), 0)
    y_grid = np.expand_dims(np.expand_dims(y_grid, 2), 0)
    z_grid = np.expand_dims(np.expand_dims(z_grid, 2), 0)
    mesh = np.vstack((x_grid, y_grid, z_grid))
    return mesh

In [58]:
def prepare_vector(vector_x, vector_y, vector_z):
    r"""
    """
    msg = 'Vector should be on a 2D plane.'
    assert len(vector_x.shape) == len(vector_y.shape) ==  len(vector_z.shape) == 2, msg 
    msg = 'Vector components do not have same dimensions.' 
    assert vector_x.shape[0] == vector_y.shape[0] == vector_z.shape[0], msg
    assert vector_x.shape[1] == vector_y.shape[1] == vector_z.shape[1], msg
    vector_x = np.expand_dims(np.expand_dims(vector_x, 2), 0)
    vector_y = np.expand_dims(np.expand_dims(vector_y, 2), 0)
    vecotr_z = np.expand_dims(np.expand_dims(vector_z, 2), 0)
    vector = make_obj_array([vector_x, vector_y, vector_z])
    return vector

In [59]:
def prepare_scalar(scalar):
    r"""
    """
    msg = 'Scalar should be on a 2D plane.'
    assert len(scalar.shape) == 2, msg 
    scalar = np.expand_dims(np.expand_dims(scalar, 2), 0)
    return scalar

In [60]:
def write_to_structured_grid(file_name, data, label, mesh):
    r"""
    """
    write_structured_grid(file_name, 
                          mesh,
                          point_data=[(label, data)])

In [61]:
n_resampled, x_grid, y_grid = resample_on_structutred_grid(n_measurements, 0)

In [65]:
mesh = prepare_mesh(x_grid, y_grid, 0.25)

In [66]:
n_prepared = prepare_scalar(n_resampled)

In [68]:
write_to_structured_grid('../output/density2.vts', n_prepared, 'n', mesh)

In [69]:
print 'mesh', mesh.shape
print 'data', n_prepared.shape 

mesh (3, 100, 100, 1)
data (1, 100, 100, 1)


In [138]:
t_resampled, x_grid, y_grid = resample_on_structutred_grid(t_measurements, 0, to_clip=[0, 94, 0, 94])

In [139]:
mesh = prepare_mesh(x_grid, y_grid, 0.25)

In [140]:
t_prepared = prepare_scalar(t_resampled)

In [141]:
write_to_structured_grid('../output/temperatur6.vts', t_prepared, 't', mesh)

In [142]:
t_resampled.shape

(94, 94)

In [149]:
b_x_resampled, x_grid, y_grid = resample_on_structutred_grid(b_x_measurements, 0)
b_y_resampled, x_grid, y_grid = resample_on_structutred_grid(b_x_measurements, 0)
b_z_resampled, x_grid, y_grid = resample_on_structutred_grid(b_x_measurements, 0)

In [150]:
np.where(np.isnan(b_x_resampled))

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  8,
         8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11,
        12, 12, 13, 91, 92, 92, 93, 93, 93, 94, 94, 94, 94, 95, 95, 95, 95,
        95, 96, 96, 96, 96, 96, 96, 97, 97, 97, 97, 97, 97, 97, 98, 98, 98,
        98, 98, 98, 98, 98]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0,  1,  2,
         3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  0,  1,  2,  3,  4,  5,  6,
         7,  8,  9, 10, 11,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,
         1,  2,  3,  4,  5,  6,  7,  8,  9,  0,  1,  2,  3,  4,  5,  6,  7,
         8,  0,  1,  2,  3,  4,  5,  6,  7,  0,  1,  2,  3

In [148]:
b_x_resampled

(array([[             nan,              nan,              nan, ...,
           1.19409625e-04,   1.16660935e-04,   1.13912246e-04],
        [             nan,              nan,              nan, ...,
          -9.74559990e-06,  -3.52371282e-05,  -6.07286565e-05],
        [             nan,              nan,              nan, ...,
          -2.07129267e-04,  -2.32620795e-04,  -2.58112324e-04],
        ..., 
        [             nan,              nan,              nan, ...,
          -1.73300701e-04,  -1.38142752e-04,  -1.02984803e-04],
        [             nan,              nan,              nan, ...,
          -1.75282231e-04,  -1.46369746e-04,  -1.21999279e-04],
        [             nan,              nan,              nan, ...,
          -1.71169648e-04,  -1.46799181e-04,  -1.22428714e-04]]),
 array([[-0.032     , -0.03142424, -0.03084849, ...,  0.02327273,
          0.02384849,  0.02442424],
        [-0.032     , -0.03142424, -0.03084849, ...,  0.02327273,
          0.02384849,  0